In [3]:
!unzip -d ./ ../alexa-prize-topical-chat-dataset/conversations.zip
!unzip -d ./ ../alexa-prize-topical-chat-dataset/reading_sets.zip

### assign entity to each utterance
The entity need to be post-processed

In [4]:
import json
import copy
import pandas as pd
topic = {1:'Fashion',2:'Politics', 3:'Books', 4:'Sports',5:'General Entertainment',6:'Music',7:'Science & Technology', 8:'Movie'}
assignment = pd.read_csv('entity_topic_assign.csv')
entity_assign = dict()
for index, row in assignment.iterrows():
    tmp = row['topic']
    tmp = topic[int(tmp)]
    entity_assign[row['entity']] = tmp


src1 = 'conversations/train.json'
src2 = 'reading_sets/post-build/train.json'
with open(src1,'r') as f, open(src2,'r') as g:
    t1 = json.load(f)
    t2 = json.load(g)
dataset = []
print('Total number of dialog: ',len(t1.keys()))
num1=0
for i in t1.keys():
    num1 = num1+1
    t1[i].pop("article_url")
    t1[i].pop('config')
    t1[i].pop('conversation_rating')
    for j in range(len(t1[i]['content'])):
        agent = t1[i]['content'][j]['agent']
        source = t1[i]['content'][j]['knowledge_source']
        t1[i]['content'][j]['entity'] = []
        t1[i]['content'][j]['topic'] = []
        source = set(source)
        set_fs = set(['FS1','FS2','FS3'])
        set_as = set(['AS1','AS2','AS3'])
        if set_fs&source!=set():
            for k in list(set_fs&source):
                t1[i]['content'][j]['entity'].append(t2[i][agent][k]['entity'])
        elif set_fs&source==set() and set_as&source!=set():
            for k in list(set_as&source):
                tmp = t2[i]['article'][k]
                t1[i]['content'][j]['entity'].append("No entity: id: "+i+'; '+k+': '+tmp)
    for v in range(len(t1[i]['content'])):
        num = v
        if t1[i]['content'][v]['entity']==[]:
            set1 = set(t1[i]['content'][num-1]['knowledge_source'])&set(['FS1','FS2','FS3'])
            if num<len(t1[i]['content'])-1:
                set2 = set(t1[i]['content'][num+1]['knowledge_source'])&set(['FS1','FS2','FS3'])
            else:
                set2 = set(t1[i]['content'][num-1]['knowledge_source'])&set(['FS1','FS2','FS3'])
            if len(set1&set2)!=0:
                source = [x for x in set1&set2]
                for u in range(len(source)):
                    t1[i]['content'][v]['entity'].append(t2[i][t1[i]['content'][v]["agent"]][source[u]]['entity'])
                t1[i]['content'][v]['entity'].append('INFER')
            else:
                tmp = []
                source = [x for x in set1|set2]
                if len(source)>0:
                    for u in range(len(source)):
                        tmp.append(t2[i][t1[i]['content'][v]["agent"]][source[u]]['entity'])
                    t1[i]['content'][v]['entity'] = ["No entity: topic change",tmp]
                else:
                    t1[i]['content'][v]['entity'] = ["No entity: cannot infer from former and latter"]
    dataset.append([])
    dataset[-1] = copy.deepcopy(t1[i])

Total number of dialog:  8628


### label topics with Rule 1, 2, 3, 4, 6. 

In [5]:
import re
# for utterance without topics, the assign the name entity to that appear the most. If a dialog don't have topic, assign all topics of entities
for i in range(len(dataset)):
    tmp = {}
    for k in range(len(dataset[i]['content'])):
        if 'No entity' in dataset[i]['content'][k]['entity'][0]:
            continue
        for j in range(len(dataset[i]['content'][k]['entity'])):
            if dataset[i]['content'][k]['entity'][j]=='INFER':
                pass
            else:
                #print(dataset[i]['content'][k]['entity'])
                try:
                    dataset[i]['content'][k]['topic'].append(entity_assign[dataset[i]['content'][k]['entity'][j]])
                except:
                    print(dataset[i]['content'][k]['entity'])
                if entity_assign[dataset[i]['content'][k]['entity'][j]] not in tmp:
                    tmp[entity_assign[dataset[i]['content'][k]['entity'][j]]] = 1
                else:
                    tmp[entity_assign[dataset[i]['content'][k]['entity'][j]]] = tmp[entity_assign[dataset[i]['content'][k]['entity'][j]]]+1
    if tmp!={}:
        max_key = max(tmp, key=tmp.get)
        keys = []
        for v in tmp:
            if tmp[v]==tmp[max_key]:
                keys.append(v)
    else:
        keys = []
        for k in range(len(dataset[i]['content'])):
            if 'No entity: id: ' in dataset[i]['content'][k]['entity'][0]:
                idx = re.findall(r'No entity: id: (.+); AS',dataset[i]['content'][k]['entity'][0])
        for tt in ['FS1','FS2','FS3']:
            keys.append(entity_assign[t2[idx[0]]['agent_1'][tt]['entity']])
        '''
        print(i)
        print(json.dumps(dataset[i]['content'],indent=4))
        max_key = max(tmp, key=tmp.get)
        keys = []
        for v in tmp:
            if v==max_key:
                keys.append(v)
        '''
    for k in range(len(dataset[i]['content'])):
        if dataset[i]['content'][k]['topic']==[]:
            if dataset[i]['content'][k]['entity'][0]=='No entity: topic change': 
                for j in dataset[i]['content'][k]['entity'][1]:
                    dataset[i]['content'][k]['topic'].append(entity_assign[j])
            else:
                for v in keys:
                    dataset[i]['content'][k]['topic'].append(v)

### Save csv file and reload them to list. Check bugs for file encoding. 

In [6]:
sentence = []
NER = []
topic = []
for i in dataset:
    for j in i['content']:
        sentence.append(j['message'])
        NER.append(j['entity'])
        topic.append(j['topic'])
data = {'sentence':sentence,'entity':NER,'topic':topic}
df = pd.DataFrame(data)  
df.to_csv('train.csv',encoding='utf-8')

train = pd.read_csv('train.csv', index_col=0)

train = train[['sentence','entity','topic']]

import ast
all_entity = train['entity'].values.tolist()
for i in range(len(all_entity)):
    try:
        tmp = copy.copy(all_entity[i])
        all_entity[i] = ast.literal_eval(all_entity[i])
    except:
        print(tmp)

all_topic = train['topic'].values.tolist()
p = 0
for i in range(len(all_topic)):
    try:
        p = p+1
        tmp = copy.copy(all_topic[i])
        all_topic[i] = ast.literal_eval(all_topic[i])
    except:
        print(p)
        print(tmp)

message = train['sentence'].values.tolist()

### Label topics with Rule 5

In [7]:
train['topic_with_general'] = None
train['topic_as'] = None
topic_as = []
number3 = 0 # count the number of adjusting AS to previous appeared entity
number4 = 0 # count the number of all AS
for i in range(len(all_entity)):
    tmp = all_entity[i]
    t1 = True
    t2 = True
    if 'No entity: id: ' in tmp[0]:
        number4+=1
        k1 = i-1
        k2 = i+1
        if 'No entity: cannot infer from former and latter' in all_entity[k1][0] or 'No entity: id: ' in all_entity[k1][0]:
            t1 = False
        if 'No entity: cannot infer from former and latter' in all_entity[k2][0] or 'No entity: id: ' in all_entity[k2][0]:
            t2 = False
        if t1 or t2:
            number3+=1
            topic_as.append(list(set(all_topic[k1]+all_topic[k2])))
            #print(list(set(all_topic[k1]+all_topic[k2])))
            #print(tmp)
            #print(all_entity[k1])
            #print(all_entity[k2])
        else:
            topic_as.append(list(set(all_topic[i])))
            #print(list(set(all_topic[k1]+all_topic[k2])))
            #print(tmp)
            #print(all_entity[k1])
            #print(all_entity[k2])
    else:
        topic_as.append(list(set(all_topic[i])))

#### There are 6997 utterances are label with Rule 5

In [8]:
number3,number4 # count the number of adjusting AS to previous appeared entity, count the number of all AS

(6997, 14279)

### Label general topic

get the first utterance and last 2 utterances index

In [9]:
import json
import copy
import pandas as pd
src1 = 'conversations/train.json'
src2 = 'reading_sets/post-build/train.json'
with open(src1,'r') as f, open(src2,'r') as g:
    t1 = json.load(f)
    t2 = json.load(g)
num1 = -1
num2 = []
for i in t1.keys():
    #num1 = num1+1
    #print(t1[i])
    t1[i].pop("article_url")
    t1[i].pop('config')
    for j in range(len(t1[i]['content'])):
        num1 = num1+1
        if j<1 or j>(len(t1[i]['content'])-3):
            num2.append(num1)
num2 = set(num2)

group utterances in conversations

In [10]:
import json
import copy
import pandas as pd
src1 = 'conversations/train.json'
src2 = 'reading_sets/post-build/train.json'
with open(src1,'r') as f, open(src2,'r') as g:
    t1 = json.load(f)
    t2 = json.load(g)
s1 = -1
s2 = []
for i in t1.keys():
    #num1 = num1+1
    #print(t1[i])
    t1[i].pop("article_url")
    t1[i].pop('config')
    temp = []
    for j in range(len(t1[i]['content'])):
        s1 = s1+1
        #if j<1 or j>(len(t1[i]['content'])-3):
        temp.append(s1)
    s2.append([])
    s2[-1] = copy.copy(temp)

#### Label general topic 

In [11]:
topic_with_general = []
change = []
ext = {'Fashion':['fashion'],
       'Politics':['politics','us','united state','vote','republican','govern',"gov't",'obama','leader','senate','summit','democratic'], 
       'Books':['book','shakespeare','isaac asimov','robot','read','stan lee','libraries','library','hero','character','novel'], 
       'Sports':['sport','nba'],
       'General Entertainment':['game','entertain'],
       'Music':['rap','sing','song','music'],
       'Science & Technology':['ocean','water','earth','programming language'], 
       'Movie':['movie','imdb','superhero','actor','Oscars','film']}
for i in range(len(all_entity)):
    if i in num2:
        if 'No entity: cannot infer from former and latter' in all_entity[i][0] or 'No entity: id: ' in all_entity[i][0]:
            topic_with_general.append(topic_as[i])
        elif 'No entity: topic change'==all_entity[i][0]:
            check = False
            tmp_sent = message[i].lower()
            # {1:'Fashion',2:'Politics', 3:'Books', 4:'Sports',5:'General Entertainment',6:'Music',7:'Science & Technology', 8:'Movie'}
            ext_temp = []
            for x in topic_as[i]:
                ext_temp = ext_temp+ext[x]
            for j in all_entity[i][1]+ext_temp:
                j = j.lower()
                j=j.split('(')[0].strip()
                temp = j.split()
                temp = list(filter(lambda y:y!='',temp))
                for u in temp:
                    if u in tmp_sent:
                        check = True
            if check==False:
                topic_with_general.append(['General'])
                change.append(i)
            else:
                topic_with_general.append(topic_as[i])
        else:
            check = False
            tmp_sent = str(message[i]).lower()
            ext_temp = []
            for x in topic_as[i]:
                ext_temp = ext_temp+ext[x]
            for j in all_entity[i]+ext_temp:
                j = j.lower()
                j=j.split('(')[0].strip()
                temp = j.split()
                temp = list(filter(lambda y:y!='',temp))
                for u in temp:
                    if u in tmp_sent:
                        check = True
            if check==False:
                topic_with_general.append(['General'])
                change.append(i)
            else:
                topic_with_general.append(topic_as[i])
    else:
        topic_with_general.append(topic_as[i])

#### Save middle result to train_v4.csv. The topic label is ready to use

In [12]:
train['topic_as'] = topic_as
train['topic_with_general'] = topic_with_general

train.to_csv('train_v4.csv')

### post process entitiy label

#### post process rule 2 and rule 3

In [13]:
message_en=[]
entity_en=[]
topic_en=[]
topic_as_en=[]
topic_general_en=[]
check = []
rule2 = []
for i in s2:
    for j in i:
        if 'No entity: topic change' in all_entity[j][0]:
            check.append(j)
            message_en.append(copy.copy(message[j]))
            entity_en.append(list(set(all_entity[j][1])))
            topic_en.append(copy.copy(all_topic[j]))
            topic_as_en.append(copy.copy(topic_as[j]))
            topic_general_en.append(copy.copy(topic_with_general[j]))
        elif 'INFER' in all_entity[j]:
            check.append(j)
            rule2.append(j)
            message_en.append(copy.copy(message[j]))
            entity_en.append(copy.copy(all_entity[j][:-1]))
            topic_en.append(copy.copy(all_topic[j]))
            topic_as_en.append(copy.copy(topic_as[j]))
            topic_general_en.append(copy.copy(topic_with_general[j]))
        else:
            check.append(j)
            message_en.append(copy.copy(message[j]))
            entity_en.append(copy.copy(all_entity[j]))
            topic_en.append(copy.copy(all_topic[j]))
            topic_as_en.append(copy.copy(topic_as[j]))
            topic_general_en.append(copy.copy(topic_with_general[j]))

#### post process rule 4,5,6

Caculate entity frequency in each conversation

In [14]:
entity_freq = [[] for i in range(len(s2))]
entity_except = []
for i in range(len(s2)):
    tmp = {}
    for j in s2[i]:
        if 'No entity: ' not in all_entity[j][0] and 'INFER' != all_entity[j][-1]:
            for k in all_entity[j]:
                if k not in tmp:
                    tmp[k] = 1
                if k in tmp:
                    tmp[k]+=1
    if tmp=={}:
        entity_except.append(i)
        entity_freq[i] = []
    else:
        max_key = max(tmp, key=tmp.get)
        keys = []
        for v in tmp:
            if tmp[v]==tmp[max_key]:
                keys.append(v)
        entity_freq[i] = keys

#### implement post process for rule 4, 5, 6

In [15]:
check_rule4_0 = []
check_rule4_1 = [[],[]]
check_rule3 = []
entity2 = copy.deepcopy(entity_en)
number3 = 0 # count the number of adjusting AS to previous appeared entity
number4 = 0 # count the number of all AS
for i in range(len(entity_en)):
    tmp = all_entity[i]
    t1 = True
    t2 = True
    if 'No entity: id: ' in tmp[0]:
        number4+=1
        k1 = i-1
        k2 = i+1
        if 'No entity: cannot infer from former and latter' in all_entity[k1][0] or 'No entity: id: ' in all_entity[k1][0]:
            t1 = False
        if 'No entity: cannot infer from former and latter' in all_entity[k2][0] or 'No entity: id: ' in all_entity[k2][0]:
            t2 = False
        if t1 or t2:
            # Rule5
            number3+=1
            check_rule4_1[0].append(i)
            
            temp2 = list(set(entity2[k1]+entity2[k2]))
            temp2 = list(filter(lambda x:'No entity:'not in x,temp2))
            entity2[i] = copy.copy(temp2)
        else:
            # Rule6
            check_rule4_1[1].append(i)
            for idx, diag in enumerate(s2):
                if i in diag:
                    entity2[i] = copy.copy(entity_freq[idx])
                    break
    elif 'No entity: cannot infer from former and latter' in tmp[0]:
        # Rule4
        check_rule4_0.append(i)
        for idx, diag in enumerate(s2):
            if i in diag:
                entity2[i] = copy.copy(entity_freq[idx])
                break
    elif 'No entity: topic change' in tmp[0]:
        # Rule3
        check_rule3.append(i)

### Save as a whole dataset

In [16]:
#data = {'sentence':message_en,'entity_ori':all_entity,'entity':entity2,'topic':topic_as_en,'topic_with_general':topic_general_en}
data = {'sentence':message_en,'entity':entity2,'topic_with_general':topic_general_en}
pd.DataFrame(data).to_csv('train_final.csv')

### Save dataset according to their rule's

In [17]:
len(rule2)+len(check_rule3)+len(check_rule4_0)+len(check_rule4_1[0])+len(check_rule4_1[1])

41864

In [18]:
rule1 = []
infer_set = set(rule2+check_rule3+check_rule4_0+check_rule4_1[0]+check_rule4_1[1])
for i in range(len(all_entity)):
    if i not in infer_set:
        rule1.append(i)

In [19]:
len(rule1),len(rule1)+len(rule2)+len(check_rule3)+len(check_rule4_0)+len(check_rule4_1[0])+len(check_rule4_1[1])

(146514, 188378)

In [20]:
idx = [rule1, rule2, check_rule3, check_rule4_0, check_rule4_1[0], check_rule4_1[1]]

#### Rule 1

In [21]:
idx1 = idx[0]
message1 = []
entity1 = []
topic1 = []
for i in idx1:
    message1.append(message_en[i])
    entity1.append(entity2[i])
    topic1.append(topic_general_en[i])
data = {'id':idx1,'sentence':message1,'entity':entity1,'topic':topic1}
pd.DataFrame(data).to_csv('train_final_rule1.csv')

#### Rule 2

In [22]:
idx1 = idx[1]
message1 = []
entity1 = []
topic1 = []
for i in idx1:
    message1.append(message_en[i])
    entity1.append(entity2[i])
    topic1.append(topic_general_en[i])
data = {'id':idx1,'sentence':message1,'entity':entity1,'topic':topic1}
pd.DataFrame(data).to_csv('train_final_rule2.csv')


#### Rule 3

In [23]:
idx1 = idx[2]
message1 = []
entity1 = []
topic1 = []
for i in idx1:
    message1.append(message_en[i])
    entity1.append(entity2[i])
    topic1.append(topic_general_en[i])
data = {'id':idx1,'sentence':message1,'entity':entity1,'topic':topic1}
pd.DataFrame(data).to_csv('train_final_rule3.csv')

#### Rule 4

In [24]:
idx1 = idx[3]
message1 = []
entity1 = []
topic1 = []
for i in idx1:
    message1.append(message_en[i])
    entity1.append(entity2[i])
    topic1.append(topic_general_en[i])
data = {'id':idx1,'sentence':message1,'entity':entity1,'topic':topic1}
pd.DataFrame(data).to_csv('train_final_rule4.csv')

#### Rule 5

In [25]:
idx1 = idx[4]
message1 = []
entity1 = []
topic1 = []
for i in idx1:
    message1.append(message_en[i])
    entity1.append(entity2[i])
    topic1.append(topic_general_en[i])
data = {'id':idx1,'sentence':message1,'entity':entity1,'topic':topic1}
pd.DataFrame(data).to_csv('train_final_rule5.csv')

#### Rule 6

In [26]:
idx1 = idx[5]
message1 = []
entity1 = []
topic1 = []
for i in idx1:
    message1.append(message_en[i])
    entity1.append(entity2[i])
    topic1.append(topic_general_en[i])
data = {'id':idx1,'sentence':message1,'entity':entity1,'topic':topic1}
pd.DataFrame(data).to_csv('train_final_rule6.csv')

# Use word2vec embedding similarity to filter out labels that are not trustable
#### This code propose another entity and topic labeling method using RAKE and word2vec embedding. Only labels that are agreed with both methods will be saved. 
We use word2vec embeddings from https://github.com/plasticityai/magnitude

In [49]:
!mkdir filter

mkdir: cannot create directory ‘filter’: File exists


In [ ]:
!pip install pymagnitude==0.1.143
!pip install rake-nltk==1.0.4

In [50]:
!cp train_final_rule*.csv filter/

In [51]:
!mkdir filter/word2vec;cd filter/word2vec; wget http://magnitude.plasticity.ai/word2vec/medium/GoogleNews-vectors-negative300.magnitude

wget: missing URL
Usage: wget [OPTION]... [URL]...

Try `wget --help' for more options.


#### Rule2

In [ ]:
!cd filter;python keywords_ext.py train_final_rule2.csv r2_keywords.txt;python gold_label_classifier.py train_final_rule2.csv r2_keywords.txt r2_topic_agreement.csv r2_entity_agreement.csv ./word2vec/GoogleNews-vectors-negative300.magnitude

#### Rule 3

In [ ]:
!cd filter;python keywords_ext.py train_final_rule3.csv r3_keywords.txt;python gold_label_classifier.py train_final_rule3.csv r3_keywords.txt r3_topic_agreement.csv r3_entity_agreement.csv ./word2vec/GoogleNews-vectors-negative300.magnitude

#### Rule 4

In [ ]:
!cd filter;python keywords_ext.py train_final_rule4.csv r4_keywords.txt;python gold_label_classifier.py train_final_rule4.csv r4_keywords.txt r4_topic_agreement.csv r4_entity_agreement.csv ./word2vec/GoogleNews-vectors-negative300.magnitude

#### Rule 5

In [ ]:
!cd filter;python keywords_ext.py train_final_rule5.csv r5_keywords.txt;python gold_label_classifier.py train_final_rule5.csv r5_keywords.txt r5_topic_agreement.csv r5_entity_agreement.csv ./word2vec/GoogleNews-vectors-negative300.magnitude

#### Rule 6

In [ ]:
!cd filter;python keywords_ext.py train_final_rule6.csv r6_keywords.txt;python gold_label_classifier.py train_final_rule6.csv r6_keywords.txt r6_topic_agreement.csv r6_entity_agreement.csv ./word2vec/GoogleNews-vectors-negative300.magnitude